In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
path = '/home/kunal/Downloads/lord_of_machines/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
campaign = pd.read_csv(path +'campaign_data.csv')

In [39]:
train_input = pd.read_csv(path + 'impact_encoded_train.csv')
test_input = pd.read_csv(path + 'impact_encoded_test.csv')

### CREATE EXTRA FEATURES USING GROUPBY STATISTICS

In [3]:
train = train.merge(campaign, on ='campaign_id',how = 'left')
test = test.merge(campaign, on ='campaign_id',how = 'left')

In [ ]:
#x1 = train.groupby('campaign_id')['is_click'].mean().sort_values(ascending = False).values

In [ ]:
#x3 = train.groupby('campaign_id')['is_open'].mean().sort_values(ascending = False).values

In [ ]:
#x2 = train['campaign_id'].value_counts().values

In [4]:
#def get_time(cell):
#    cell = cell.split(' ')[-1]
#    cell = cell.split(':')[0]
#    return cell
#train['hour'] = train['send_date'].apply(get_time)

In [ ]:
#train[train['is_click'] == 1].hour.value_counts()

In [ ]:
#x5 = train['hour'].value_counts().values

In [ ]:
#test['hour'] = test['send_date'].apply(get_time)

In [ ]:
#test['hour'].value_counts()

In [ ]:
train_input = pd.read_csv(path + 'impact_encoded_train.csv')
test_input = pd.read_csv(path + 'impact_encoded_test.csv')

In [ ]:
#x3 = train.groupby('hour')['is_open'].mean().sort_values(ascending = False).values

In [5]:
## Day of the week
#def get_date(cell):
#    return cell.split(' ')[0]
#train['date'] = train['send_date'].apply(get_date)

In [ ]:
#import datetime
#exp = train['date'][0]

In [6]:
#from datetime import datetime

In [7]:
#def get_weekday(cell):
#    return datetime.strptime(cell,'%d-%m-%Y').weekday()

In [8]:
#train['weekday'] = train['date'].apply(get_weekday)

In [ ]:
#train['weekday'].value_counts()

In [9]:
#test['date'] = test['send_date'].apply(get_date)
#test['weekday'] = test['date'].apply(get_weekday)

In [ ]:
tr.groupby('weekday')['is_open'].mean()

In [ ]:
train.groupby('hour')['is_open'].mean()

In [30]:
campaign['len_sub'] = campaign['subject'].str.split(' ').apply(len)

In [20]:
train['len_sub'] = train['campaign_id'].map(pd.Series(campaign['len_sub'],index = campaign['campaign_id']))
test['len_sub'] = test['campaign_id'].map(pd.Series(campaign['len_sub'],index = campaign['campaign_id']))

In [31]:
campaign['len_sub_email'] = campaign['email_body'].str.split(' ').apply(len)

In [24]:
train['len_sub_email'] = train['campaign_id'].map(pd.Series(campaign['len_sub_email'],index = campaign['campaign_id']))
test['len_sub_email'] = test['campaign_id'].map(pd.Series(campaign['len_sub_email'],index = campaign['campaign_id']))

In [33]:
train.drop(['len_sub','len_sub_email'],axis = 1,inplace = True)
test.drop(['len_sub','len_sub_email'],axis = 1,inplace = True)

In [34]:
train = train.merge(campaign[['campaign_id','len_sub','len_sub_email']],on = 'campaign_id',how = 'left')
test = test.merge(campaign[['campaign_id','len_sub','len_sub_email']],on = 'campaign_id',how = 'left')

In [41]:
train_input['len_sub'] = train['len_sub']
train_input['len_sub_email'] = train['len_sub_email']

test_input['len_sub'] = test['len_sub']
test_input['len_sub_email'] = test['len_sub_email']

In [43]:
train_input.to_csv(path + 'impact_encoded_train.csv',index = False)

In [44]:
test_input.to_csv(path + 'impact_encoded_test.csv',index = False)

In [ ]:
# This way we have randomness and are able to reproduce the behaviour within this cell.
np.random.seed(13)
from sklearn.model_selection import KFold

def impact_coding(data, feature, target='y'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
    
    In this implementation the KFolds use shuffling. If you want reproducibility the cv 
    could be moved to a parameter.
    '''
    n_folds = 10
    n_inner_folds = 5
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature]):
            impact_coded_cv = pd.Series()
            kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
            inner_split = 0
            inner_oof_mean_cv = pd.DataFrame()
            oof_default_inner_mean = data.iloc[infold][target].mean()
            for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
                # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
                oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
                impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                            lambda x: oof_mean[x[feature]]
                                      if x[feature] in oof_mean.index
                                      else oof_default_inner_mean
                            , axis=1))

                # Also populate mapping (this has all group -> mean for all inner CV folds)
                inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
                inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
                inner_split += 1

            # Also populate mapping
            oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
            oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
            split += 1
            
            impact_coded = impact_coded.append(data.iloc[oof].apply(
                            lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                      if x[feature] in inner_oof_mean_cv.index
                                      else oof_default_mean
                            , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

In [ ]:
f = 'weekday'

In [ ]:
# Apply the encoding to training and test data, and preserve the mapping
impact_coding_map = {}
print("Impact coding for {}".format(f))
train["impact_encoded_{}".format(f)], impact_coding_mapping, default_coding = impact_coding(train, f,'is_click')
impact_coding_map[f] = (impact_coding_mapping, default_coding)
mapping, default_mean = impact_coding_map[f]
test["impact_encoded_{}".format(f)] = test.apply(lambda x: mapping[x[f]]
                                                                     if x[f] in mapping
                                                                     else default_mean
                                                           , axis=1)

In [ ]:
impact_coding_map = {}
print("Impact coding for {}".format(f))
train["impact_encoded_open_{}".format(f)], impact_coding_mapping, default_coding = impact_coding(train, f,'is_open')
impact_coding_map[f] = (impact_coding_mapping, default_coding)
mapping, default_mean = impact_coding_map[f]
test["impact_encoded_open_{}".format(f)] = test.apply(lambda x: mapping[x[f]]
                                                                     if x[f] in mapping
                                                                     else default_mean
                                                           , axis=1)